In [36]:
from __future__ import division #used in Python2.x so the division btw integers result in float, default in python3
from pyomo.environ import * #library for modeling and optimisation
import argparse
from pyomo.opt import SolverStatus, TerminationCondition #provide status of solvers
import pandas as pd #data manipulation library
import numpy as np 
import pyutilib.services
import pickle
import random
import sqlite3
import copy


import warnings
warnings.filterwarnings('ignore')

Results_folder = "./results/"

INTEREST = 0.03
SCENARIOS = 1
SALV_PRICE = 0
    
SALVAGE = ""

OPT="_NPV"

OPT = OPT + "_"+str(SALV_PRICE) #QUESTION: Here OPT becomes _NPV_0 > Why do we need that?

#REQUIRES CBC TO BE INSTALLED ON THE MACHINE USED!!!  See https://github.com/coin-or/Cbc -- OTHER SOLVERS CAN BE USED -- 
#THEN CHANGE this line: "opt = SolverFactory('cbc') #Here we use the cbc solver -- open source software"

path = "C:/Users/juhu/OneDrive - Norwegian University of Life Sciences/Documents/Python/Simple_Problem/"#ADJUST TO OWN PATH #"c:/mytemp/avohakkutpois/Files_for_optimization/temp/"


class optimization:
    def __init__(self):
        #c = 0
        # Load data and preprocess by extracting branch and id from the id column
        data_opt =  pd.read_csv(path+"Data_Juliette.csv")
        data_opt['branch'] = [int(str(i)[-2:]) for i in data_opt['id']]
        data_opt['id'] = [int(str(i)[:-2]) for i in data_opt['id']]

        for k in range(0,SCENARIOS):
            if k == 0:
                dat = data_opt
                dat['iteration']=k
        data_opt = dat
        combinations = 1

        #CREATE replicates with varying iterations
        # Data structure initialization
        all_data = data_opt

        #Index_values = all_data.set_index(['id','branch']).index.unique() #array with unique index values composed of 'id' and 'branch'
        all_data = all_data.set_index(['id','branch','year','iteration']) #reassigns the Df with new multi-level index
        #AREA = all_data.loc[slice(None),0,2016,all_data.index.get_level_values(3).min()]['AREA']
        all_data = all_data.fillna(0)
        all_data['year'] = all_data.index.get_level_values(2)
        self.data_opt=all_data #the preprocessing fgets embedded in the class

        self.combinations = 1

        #CREATE replicates with varying iterations

        self.all_data = self.data_opt
        self.Index_values = self.all_data.drop(['year'], axis=1).reset_index().set_index(['id','branch']).index.unique()#all_data.set_index(['id','branch']).index.unique()
        self.AREA = self.all_data.loc[slice(None),0,2016,self.all_data.index.get_level_values(3).min()]['AREA']
        self.all_data = self.all_data.fillna(0)

        self.createModel()

    def createModel(self):
        # Declare sets - These used to recongnize the number of stands, regimes and number of periods in the analysis.
        # Define sets, variables, and constraints for the optimization problem
        
        self.model1 = ConcreteModel() #defining the model

        self.model1.stands = Set(initialize = list(set(self.all_data.index.get_level_values(0)))) #initialized with unique values from the first level of the index created earlier
        self.model1.year = Set(initialize = list(set(self.all_data.index.get_level_values(2)))) #initialized with unique values from the third level of the index created earlier
        self.model1.iteration = Set(initialize = list(set(self.all_data.index.get_level_values(3))))        
        self.model1.regimes = Set(initialize = list(set(self.all_data.index.get_level_values(1))))
        self.model1.scen_index = Set(initialize= [i for i in range(0,self.combinations)])
        self.model1.Index_values = self.Index_values

        # Indexes (stand, regime)-- excludes those combinations that have no regimes simulated

        def index_rule(model1):
            index = []
            for (s,r) in model1.Index_values: #stand_set
                index.append((s,r))
            return index
        self.model1.index1 = Set(dimen=2, initialize=index_rule)

        #Decision variable
        self.model1.X1 = Var(self.model1.index1, within=NonNegativeReals, bounds=(0,1), initialize=1)

        self.all_data['year'] = self.all_data.index.get_level_values(2)

        #objective function:
        def outcome_rule(model1):
            return sum((self.all_data.Harvested_V.loc[(s,r,k,it)]*self.all_data.AREA.loc[(s,r,k,it)]* self.model1.X1[(s,r)])/((1+INTEREST)**(2.5+self.all_data.year[(s,r,k,it)]))  for (s,r) in self.model1.index1 for k in self.model1.year for it in self.model1.iteration)
        self.model1.OBJ = Objective(rule=outcome_rule, sense=maximize)

        #Constraint:
        def regime_rule(model1, s):
            row_sum = sum(model1.X1[(s,r)] for r in [x[1] for x in model1.index1 if x[0] == s])
            return row_sum == 1
        self.model1.regime_limit = Constraint(self.model1.stands, rule=regime_rule)

    def solve(self):
        # Specify the solver and solve the model
        opt = SolverFactory('cbc') #Here we use the cbc solver -- open source software
        self.results = opt.solve(self.model1,tee=False) #We solve a problem, but do not show the solver output

# Create an optimization object        
t1 = optimization()
t2 = copy.deepcopy(t1)

# Modify the optimization model to focus on. Currently maximizing the NPV.

### MAX MINIMUM NPV:
#Max min
try:
    t2.model1.del_component(t2.model1.NPV_INV)
except:
    print("NONE")

#A function that evaluates NPV -- this includes wind disturbed salvage (however, the data doesn't include -- so it could be simplified.)    
t2.model1.NPV= Var(within=NonNegativeReals)
def NPV_INVENTORY(model1,it):
    row_sum = sum(((t2.all_data.income.loc[(s,r,k,it)]+t2.all_data.natural_rm_wind.loc[(s,r,k,it)]*SALV_PRICE)*t2.all_data.AREA.loc[(s,r,k,it)]* t2.model1.X1[(s,r)])/((1+INTEREST)**(2.5+t2.all_data.year[(s,r,k,it)]-2016))  for (s,r) in t2.model1.index1 for k in t2.model1.year)+sum((t2.all_data.PV.loc[(s,r,max(t2.all_data.year[(s,r,slice(None),t2.all_data.index.get_level_values(3).min())]),it)]*t2.all_data.AREA.loc[(s,r,max(t2.all_data.year[(s,r,slice(None),t2.all_data.index.get_level_values(3).min())]),it)]* t2.model1.X1[(s,r)])/((1+INTEREST)**(2.5+max(t2.all_data.year[(s,r,slice(None),t2.all_data.index.get_level_values(3).min())])-2016))  for (s,r) in t2.model1.index1)
    return t2.model1.NPV<=row_sum
t2.model1.NPV_INV= Constraint(t2.model1.iteration,rule=NPV_INVENTORY)


# #JUL:A function that evaluates the combined HSI
# t2.model1.HSI=Var(within=NonNegativeReals, bounds=(0,1))
# def COMBINED_HSI(model1, s):
#     row_prod = prod((1-t2.all_data.LESSER_SPOTTED_WOODPECKER.loc[(s,r,k)]) *(1-t2.all_data.THREE_TOED_WOODPECKER.loc[(s,r,k)])*(1-t2.all_data.LONG_TAILED_TIT.loc[(s,r,k)])* t2.model1.X1[(s,r)] for (s,r) in t2.model1.index1 for k in t2.model1.year)
#     return t2.model1.HSI <= (1 - row_prod)
# # t2.model1.COMB_HSI = Constraint(t2.model1.stands, rule=COMBINED_HSI)

# #JUL:A function that evaluates the combined HSI
# t2.model1.HSI=Var(within=NonNegativeReals, bounds=(0,1))
# def COMBINED_HSI(model1,s):
#     # Define a list of species name
#     species_name = ['LESSER_SPOTTED_WOODPECKER', 'THREE_TOED_WOODPECKER']
#     row_prod=[]
#     for species in species_name:
#         product = (1-t2.all_data[species].loc[(s,r,k)]* t2.model1.X1[(s,r)] for (s,r) in t2.model1.index1 for k in t2.model1.year)
#         row_prod.append(product)
#     final_product=prod(row_prod)
#     return t2.model1.HSI <= (1 - final_product)
# t2.model1.COMB_HSI = Constraint(t2.model1.stands, rule=COMBINED_HSI)

#JUL:Constraint increasing HSI:
def HSI_rule(model1, s):
    HSI_k = t2.all_data.HSI.loc[(s, r, k)] * t2.model1.X1[(s, r)]
    HSI_k_plus_1 = t2.all_data.HSI.loc[(s, r, k+1)] * t2.model1.X1[(s, r)]
    return HSI_k_plus_1 >= HSI_k
t2.model1.HSI_limit = Constraint(t2.model1.stands, rule=HSI_rule)


#Likely not needed -- used if constraints for even flow are present, and tries to remove them.
try:
    t2.model1.del_component(t2.model1.EVEN_inc)
    t2.model1.del_component(t2.model1.EF)
except:
    print("NONE")

#Objective function -- maximizing NPV    
def outcome_rule(model1):
    return t2.model1.NPV +t2.model1.HSI
t2.model1.OBJ = Objective(rule=outcome_rule, sense=maximize)

# Solve the modified optimization model
t2.solve()

#Function to extract decision variables from the optimized model
def GET_DECISION_DATA():
        st = []
        reg = []
        vals = []
        for (s,r) in t2.model1.index1:
            st = st+[s]
            reg = reg+[r] 
            vals = vals+[t2.model1.X1[(s,r)].value]
        data = {"id":st,"branch":reg,"value":vals}
        df= pd.DataFrame(data)
        df = df.set_index(['id','branch'])
        return df

# Extract decision data and merge with original dataset    
dec  = GET_DECISION_DATA()
merged_df = dec.merge(t2.all_data, left_index=True, right_index=True, how='left')

# Save results to a CSV file
merged_df.to_csv(path+"output2d.csv")

NONE
NONE
'pyomo.core.base.objective.ScalarObjective'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.objective.ScalarObjective'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


In [37]:
#JUL:A function that evaluates the combined HSI
t2.model1.HSIx=Var(t2.model1.stands, t2.model1.year, within=NonNegativeReals, bounds=(0,1))
def COMBINED_HSI(model1, s, k):
    ttw = (1- sum(t2.model1.X1[(s,r)].value * t2.all_data.THREE_TOED_WOODPECKER.loc[(s,r,k)] for r in [x[1] for x in t2.model1.index1 if x[0] == s]))
    ltt = (1- sum(t2.model1.X1[(s,r)].value * t2.all_data.LONG_TAILED_TIT.loc[(s,r,k)] for r in [x[1] for x in t2.model1.index1 if x[0] == s]))
    lsw = (1- sum(t2.model1.X1[(s,r)].value * t2.all_data.LESSER_SPOTTED_WOODPECKER.loc[(s,r,k)] for r in [x[1] for x in t2.model1.index1 if x[0] == s]))
    birds=1-ttw*ltt*lsw
    return t2.model1.HSIx[s,k] <= birds
t2.model1.COMB_HSIx = Constraint(t2.model1.stands,t2.model1.year, rule=COMBINED_HSI)

ERROR: Rule failed when generating expression for Constraint COMB_HSIx with
index (1024, 2016.0): AttributeError: 'Series' object has no attribute
'is_component_type'
ERROR: Constructing component 'COMB_HSIx' from data=None failed:
AttributeError: 'Series' object has no attribute 'is_component_type'
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\juhu\.conda\envs\myenv\lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\NMBU\TEMP\ipykernel_28516\4000515827.py", line 9, in <module>
    t2.model1.COMB_HSIx = Constraint(t2.model1.stands,t2.model1.year, rule=COMBINED_HSI)
  File "C:\Users\juhu\AppData\Roaming\Python\Python39\site-packages\pyomo\core\base\block.py", line 580, in __setattr__
    self.add_component(name, val)
  File "C:\Users\juhu\AppData\Roaming\Python\Python39\site-packages\pyomo\core\base\block.py", line 1185, in add_component
    val.construct(data)
  File "C:\Users\juhu\AppData\Roaming\Python\Python39\site-packages\pyomo\core\base\constraint.py", line 806, in construct
    self._setitem_when_not_present(index, rule(block, index))
  File "C:\Users\juhu\AppData\Roaming\Python\Python39\site-packages\pyomo\core\base\initializer.py", line 314, in __call__
    return self._fcn(par

In [47]:
# Test to define the combined HSI for 3 species
#k = 2036
#s=1025

ttw = (1- sum(t2.model1.X1[(s,r)].value * t2.all_data.THREE_TOED_WOODPECKER.loc[(s,r,k)] for r in [x[1] for x in t2.model1.index1 if x[0] == s]))# *(1-(t2.all_data.THREE_TOED_WOODPECKER.loc[(s,r,k)]* t2.model1.X1[(s,r)].value for r in t2.model1.regimes)) *(1-(t2.all_data.LONG_TAILED_TIT.loc[(s,r,k)]* t2.model1.X1[(s,r)].value for r in t2.model1.regimes)))
ltt = (1- sum(t2.model1.X1[(s,r)].value * t2.all_data.LONG_TAILED_TIT.loc[(s,r,k)] for r in [x[1] for x in t2.model1.index1 if x[0] == s]))
lsw = (1- sum(t2.model1.X1[(s,r)].value * t2.all_data.LESSER_SPOTTED_WOODPECKER.loc[(s,r,k)] for r in [x[1] for x in t2.model1.index1 if x[0] == s]))
birds=1-ttw*ltt*lsw
birds
print(birds, ltt,ttw,lsw)

iteration
0   -2.220446e-15
dtype: float64 iteration
0    1.0
Name: LONG_TAILED_TIT, dtype: float64 iteration
0    1.0
Name: THREE_TOED_WOODPECKER, dtype: float64 iteration
0    1.0
Name: LESSER_SPOTTED_WOODPECKER, dtype: float64


In [ ]:
#TRICKS
# Create an instance of the optimization class
optimizer = optimization()

#to see the data frames
all_data_new=optimizer.all_data
merged_df_new=merged_df

#to see the stands
t2.model1.stands.data()

In [44]:
#Landscape combined HSI
#t2.model1.HSIx=Var(t2.model1.stands, t2.model1.year, within=NonNegativeReals, bounds=(0,1))
def LAND_COMB_HSI(model1,k):
    return sum(t2.model1.HSIx[s,k] for s in t2.model1.stands) >= sum(t2.model1.HSIx[s,k-1] for s in t2.model1.stands)
t2.model1.LAND_COMB_HSIx = Constraint([x for x in t2.model1.year.data() if x != 2016], rule=LAND_COMB_HSI)

ERROR: Rule failed when generating expression for Constraint LAND_COMB_HSIx
with index 2021.0: AttributeError: 'ConcreteModel' object has no attribute
'HSIx'
ERROR: Constructing component 'LAND_COMB_HSIx' from data=None failed:
AttributeError: 'ConcreteModel' object has no attribute 'HSIx'


AttributeError: 'ConcreteModel' object has no attribute 'HSIx'